### 1)  CBoW(Continuous Bag-of-words)

$ \to $ &nbsp; 토큰 순서가 어떻든, 상관이 없이 bag으로 보자. 가방 안의 토큰은 흔들리고 있으며 순서는 상관이 없습니다. 단지, 어떤 단어가 들어있는지만 고려하며 
각 단어, 토큰들은 이미 table lookup을 통해 vector로 바뀐 상태에서의 average를 보자.

- 각 토큰들이 hi-dimensitional(300d, 3000d, ...의)하게 고차원에서 점들로 있으며, 문장에 있는 점들의 average point가 문장을 표현(의미)하는 것.

- average를 하는 부분은 DAG(Directed acyclic graph)에서 그냥 노드 하나. <br>
 average는 그냥 sum이기 때문에, 자코비안 계산도 간단.

- Continuous Bag-of-words는 Continuous Bag-of-N-gram으로 generalization도 가능.
    - named entity나 place name을 파악할 때 중요. Ex)  New York University

- 장점은 CBoW는 order(순서)를 무시함에도 성능이 좋음. 다시 말해, Text classification과 같은 문제에서 CBoW를 쓰고, 그 뒤에 layer들을 추가하여 classifier를 만들면 성능이 좋은 편.
    - cf)  작년에 build it and break it competition이라는 국제 학술 행사에서, sentiment analysis를 하는 data를 주고, classifier하는 문제에서 조경현 교수님은 Bag-of-n-gram classifier로 참여. 많은 사람들이 CNN이나 RNN으로 도전. 그리고 결과는 Bag-of-n-gram classifier의 성능이 굉장히 좋게 나옴
-  실제로 practice에서는 Text classification의 base로는 CBoW로 잡자.
    - 게다가 실제로 구현할 필요조차 없음. 페이스북 리서치팀에서 FastText라는 것을 개발해놓았다. 굉장히 효율적이다!

#### CBoW(Continuous Bag-of-words) Based Multi-Class text 

CBoW Based Multi-Class text Classifier의  DAG 관점

<img src="./image/img_Cbowclassifier.PNG" width="70%">

- 문장이 있고, 문장을 토큰 단위로 나누고,(각 토큰은 x1, x2, ... xt) 그리고 각 토큰에 대해서 동일한 weight로 Table Lookup 과정을 거치면 그러면 전부 vector가 되면서  sequence of token이 sequence of vetor가 된다.
- 이 후,sequence of vetor를 average node를 이용해서 전체 averaging한 값이 input sentence에 대해서 sentence representaion.
- averaging된 결과값은 representation space 안에서 비슷한 의미를 가진 문장들은 가까운 위치에 있으며, 다른 의미를 가진 문장들은 멀리있는 형태가 될 것이다. (Text Classification 문제를 풀기에 적합한 상태)
    - ex) sentiment analysis: 문장에 positive한 sentiment의 단어들을 많이 갖고있는 문장은 서로 가깝게, negative한 sentiment의 단어들을 많이 갖고있는 문장은 서로 가깝게.
- 그러면 이제 representation이 된 vector를 arbitrary한 sub-graph를 지나(Model), softmax classifier를 통해, 전체에 대한 classify할 수 있다.


- 정형화 된 Represenation 방법은 없다.
     - 결국 sentence represenation은 NN이나 머신러닝 관점에서보면, Universial한 representation(모든 문제를 포용하는 표현 방법)이 있다기 보다는 내가 가진 문제를 푸는 것에 가장 적합한 represenation이 무엇인지를 의미한다!
        - sentiment analysis를 하는 툴을 만들었다고 하면, representation은 sentiment analysis을 하는데 적합한 representation이 나오고
        - 만약 category classification을 하는 툴을 만들었다고 하면, representation이 category classification에 적합한 representation이 나온다.
        - 이 두 representation은 compatible할 수도 있고, 안 할 수도 있다. 다시 말해, traing 하는 모델을 어떻게 만드냐에 따라서 representation이 따라간다.

- 이제 train을 위해서, forward-pass computation을 만들고, Probability distribution이 나오게 만들었다면, 처음에 언급한 것처럼 negative log probability를 계산하게 만들고, Backpropgation을 쓰고, SGD를 쓰고, early stopping을 하면 바로 training을 할 수 있다.
    - 여기에 input으로 텍스트가 아니라 비디오 프레임을 넣으면 비디오 Classification도 가능하다. (이런 것들이 Tradational Algorithm에 비해 ML이 갖는 큰 장점이다.) 물론, CBoW만으로는 최종적으로 원하는 답일 수는 없다.(sequential 고려하지 않았으니까) 






### 2) Relation Network [Santoro et al., 2017]: Skip Bigrams

- Consider all possible pairs of tokens: $ (x_{i},x_{j}), \forall i \neq j  $
    - idea 자체는 Skip Bigram이다. 꼭 나란히 있는 두 개의 토큰일 필요는 없고 띄엄 띄엄 볼 수 있으며, 문장에서 RN은 모든 pair case를 고려
- Combine two token vectors with a neural network for each pair
    - $ f(x_{i},x_{j}) = W \phi (U_{left}e_{i} + U_{right}e_{j} ) $
    - $ \phi $ is a element-wise nonlinear function, such as tanh or ReLU
    - One subgraph in the DAG.
<img  src="./image/img_RN.PNG" width="60%">

- Considers the “relation”ship between each pair of words
    - 각 pair에 대해 Relation Neural Network를 만들어서 각 pair의 continuous vector representation을 섞어서 하나의 벡터로 나타낸다. (pair의 representation을 찾는 것이다.)
- Averages all these relationship vectors
    - $ RN(X) = {{1}\over{2N(N-1)}} \sum^{T-1}_{i=1}\sum^{T}_{j=i+1}f(x_{i},x_{j})  $
    - 그리고 이렇게 나온 벡터에 Average를 취한다.
    
- 계산량이 너무 빡세서, 보통 pair만 한다. DAG를 그리면 다음과 같고 CBoW와의 차이점은 order를 무시하는 CBoW를 쓸거냐 아니면 모든 pair-wising interaction을 생각하는 RN을 쓸거냐만 바뀌었다.

<img  src="./image/img_RNDAG.PNG" width="60%">

- 장점: CBoW가 약했던 Multi-word expression도 더 감지를 잘 할 것이다. 여전히 만족스럽지 않은 것은 모든 pair를 본다는 것.
    - cf) 한국어나 독어와 같이 S + O + V의 문장 구조면 양끝단어가 중요할 수도 있지만 영어는 S+V+O인경우에는 별로인데?의 의문이 생긴다. 따라서 이에대한 대안으로 CNN을 생각할 수 있다.
    
    

### Convolutional Networks [Kim, 2014; Kalchbrenner et al., 2015]

- Captures k-grams hierarchically

- One 1-D convolutional layer: considers all k-grams
    - $ h_{t} = \phi ( \sum_{\tau = -k/2}^{k/2}W_{\tau}e_{t+\tau} )$, resulting in $ H = (h_{1},...,h_{T}) $  
    - image에서는 보통 2d conv layer를 많이 사용하고, Text, speech, audio에서는 1d conv layer를 많이 사용.
- 1d conv layer에서는 k-gram을 hierarchical하게 gradually growing 하는 structure를 보는 것으로 생각할 수 있다. 직관에 잘 맞아 보인다.
- Fits our intuition of how sentence is understood: tokens→multi-word expressions→phrases→sentence

<img  src="./image/img_CNN.PNG" width="40%">

- 그리고 실제 Practice에서 CNN이 안쓰이는 데가 없다보니 DAG에 넣을 수 있는 노드들이 굉장히 효율적이게 구현이 잘 되어있으며, Major framework에 다 들어가있다. 또한 동시에 Hardware level에서도 구현이 잘 되어있어 GPU에서 conv를 위한 커널이 구현 되어 있고, FPGA도 convolution 하는 것도 찾을 수 있고 에이직을 만드는 곳도 있다.(?)(사용하기 좋다.)

- Recent advances
    - Multi-width convolutional layers [Kim, 2014; Lee et al., 2017] 
        - 사이즈가 달라도 가능한 것?
    - Dilated convolutional layers [Kalchbrenner et al., 2016] 
        - 나중에 말하지만 볼 수 있는 사이즈를 지수적으로 늘리는 것 (Linear 하게가 아니라)
    - Gated convolutional layers [Gehring et al., 2017]
        - 언급은 안하겠지만, 잘 쓰인다고 한다.

### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 

### Appendum:  

- FastText

    - Enriching Word Vectors with Subword Information (FastText paper)<br>
    https://arxiv.org/abs/1607.04606

    - FastText 패키지 소개<br>
    https://fasttext.cc/

-  Recent advances

    - Convolutional Neural Networks for Sentence Classification (Multi-width convolutional layers)  <br>
    https://arxiv.org/abs/1408.5882

    - Neural Machine Translation in Linear Time (Dilated convolutional layers)<br>
    https://arxiv.org/abs/1610.10099

    - Convolutional Sequence to Sequence Learning (Gated convolutional layers)
    https://arxiv.org/abs/1705.03122